<a href="https://colab.research.google.com/github/JulianaDamurie/AprendizagemDeMaquina/blob/main/lista4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
# import
import time
from scipy.io import arff
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import StandardScaler  
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.model_selection import KFold, StratifiedKFold
import math
import scipy
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager
from past.builtins import xrange
from collections import OrderedDict
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
pd.options.mode.chained_assignment = None  # default='warn'

In [48]:
class KNN:
    def __init__(self, k):
        self.k = k
        
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        
    def distance(self, X1, X2):
        distance = scipy.spatial.distance.euclidean(X1, X2)
    
    def fit_distance(self, X_test,k):
        final_output = []
        distances =  scipy.spatial.distance.cdist( self.X_train , X_test, 'euclidean')
        distances_win = []
        X_win = []

        for i in range(len(X_test)):
            d = []
            votes = []
            d_individuals = []
            x_individuals = []
            for j in range(len(self.X_train)):
                dist = np.absolute(distances[j][i])
                d.append([dist, j])
            d.sort()
            d = d[0:k]
            for d, j in d:
                votes.append(self.y_train[j])
                d_individuals.append(d)
                x_individuals.append(self.X_train[j])
            ans = Counter(votes).most_common(1)[0][0]
            final_output.append(ans)
            distances_win.append(d_individuals)
            X_win.append(x_individuals)
        return distances_win,X_win
    
    def predict(self,X_test,epsilon,k):
      final_output = []
      distances, x_win = self.fit_distance(X_test,k)

      for i in range(len(X_test)):
        d_intra, x_intra = self.fit_distance(x_win[i],2)

        calculate = distances[i][0]/(d_intra[0][1] + 0.0000000000000001)
        
        if (calculate<epsilon):
          final_output.append("false")
        else:
          final_output.append("true")
      return final_output
    def score(self, X_test, y_test):
        predictions = self.predict(X_test)
        return (predictions == y_test).sum() / len(y_test)


In [49]:
class KNN_modified:
    def __init__(self, k):
        self.k = k
        
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        
    def distance(self, X1, X2):
        distance = scipy.spatial.distance.euclidean(X1, X2)
    
    def fit_distance(self, X_test,k):
        final_output = []
        distances =  scipy.spatial.distance.cdist( self.X_train , X_test, 'euclidean')
        distances_win = []
        X_win = []

        for i in range(len(X_test)):
            d = []
            votes = []
            d_individuals = []
            x_individuals = []
            for j in range(len(self.X_train)):
                dist = np.absolute(distances[j][i])
                d.append([dist, j])
            d.sort()
            d = d[0:k]
            for d, j in d:
                votes.append(self.y_train[j])
                d_individuals.append(d)
                x_individuals.append(self.X_train[j])
            ans = Counter(votes).most_common(1)[0][0]
            final_output.append(ans)
            distances_win.append(d_individuals)
            X_win.append(x_individuals)
        return distances_win,X_win
    
    def predict(self,X_test,epsilon,k):
      final_output = []
      distances, x_win = self.fit_distance(X_test,k)
      for i in range(len(X_test)):
        d_intra, x_intra = self.fit_distance(x_win[i],2)

        sum_dist_intra = 0
        sum_dist_z = 0

        for j in range(len(d_intra)):
          sum_dist_intra += d_intra[j][1]
          sum_dist_z += distances[i][j]

        calculate = sum_dist_z/(sum_dist_intra + 0.0000000000000001)
        

        if (calculate<(epsilon)):
          final_output.append("false")
        else:
          final_output.append("true")
      return final_output
    def score(self, X_test, y_test):
        predictions = self.predict(X_test)
        return (predictions == y_test).sum() / len(y_test)


In [50]:
def sample_first_prows(data, perc):
    import pandas as pd
    return data.head(int(len(data)*(perc)))

def sample_last_prows(data, perc):
    import pandas as pd
    return data.tail(int(len(data)*(perc)))

In [51]:
# read first data set
dataset = False

if dataset:
  data, meta = arff.loadarff('/content/kc1.arff')

  df1 = pd.DataFrame(data)

  df1["defects"] = df1["defects"].astype(str).str.replace("b","")
  df1["defects"] = df1["defects"].astype(str).str.replace("'","")
  y_label = "defects"
else:
  data, meta = arff.loadarff('/content/kc2.arff')

  df1 = pd.DataFrame(data)

  df1["problems"] = df1["problems"].astype(str).str.replace("b","")
  df1["problems"] = df1["problems"].astype(str).str.replace("'","")
  df1["problems"] = df1["problems"].astype(str).str.replace("yes","true")
  df1["problems"] = df1["problems"].astype(str).str.replace("no","false")
  y_label = "problems"
df1 = df1.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

df1

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,b,t,lOCode,lOComment,lOBlank,lOCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,problems
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,1.30,1.30,2.0,2.0,2.0,2.0,1.2,1.2,1.2,1.2,1.4,false
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,true
2,415.0,59.0,50.0,51.0,1159.0,8411.31,0.01,103.53,81.24,870848.58,2.80,48380.48,359.0,35.0,9.0,10.0,47.0,106.0,692.0,467.0,106.0,true
3,230.0,33.0,10.0,16.0,575.0,3732.82,0.03,39.82,93.74,148644.06,1.24,8258.00,174.0,15.0,34.0,5.0,23.0,67.0,343.0,232.0,65.0,true
4,175.0,26.0,12.0,13.0,500.0,3123.96,0.03,29.48,105.96,92103.07,1.04,5116.84,142.0,7.0,19.0,4.0,18.0,58.0,310.0,190.0,51.0,true
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,4.0,1.0,1.0,1.0,5.0,11.61,0.50,2.00,5.80,23.22,0.00,1.29,2.0,0.0,0.0,0.0,4.0,1.0,4.0,1.0,1.0,true
518,4.0,1.0,1.0,1.0,4.0,8.00,0.67,1.50,5.33,12.00,0.00,0.67,2.0,0.0,0.0,0.0,3.0,1.0,3.0,1.0,1.0,true
519,4.0,1.0,1.0,1.0,4.0,8.00,0.67,1.50,5.33,12.00,0.00,0.67,2.0,0.0,0.0,0.0,3.0,1.0,3.0,1.0,1.0,true
520,4.0,1.0,1.0,1.0,5.0,11.61,0.67,1.50,7.74,17.41,0.00,0.97,2.0,0.0,0.0,0.0,3.0,2.0,3.0,2.0,1.0,true


In [52]:
def separar_dados(df1,classe):
  if(classe=='true'):
    n_classe = 'false'
  else:
    n_classe = 'true'

  df_true = df1[df1.iloc[:, 21].str.contains(classe, regex=False)]

  df_true = shuffle(df_true)

  df_false = df1[df1.iloc[:, 21].str.contains(n_classe, regex=False)]


  df_false = shuffle(df_false)

  # divide into class 1 and class 2

  x_df1 = np.array(df_true.iloc[:, :-1].values)
  y_df1 = np.array(df_true.iloc[:, 21].values)

  x_df2 = np.array(df_false.iloc[:, :-1].values)
  y_df2 = np.array(df_false.iloc[:, 21].values)

  return x_df1,y_df1,x_df2,y_df2

In [53]:
def optimal_number_of_clusters(wcss):
    x1, y1 = 2, wcss[0]
    x2, y2 = 20, wcss[len(wcss)-1]

    distances = []
    for i in range(len(wcss)):
        x0 = i+2
        y0 = wcss[i]
        numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
        denominator = np.sqrt((y2 - y1)**2 + (x2 - x1)**2)
        distances.append(numerator/denominator)
    
    return distances.index(max(distances)) + 2

In [54]:
skf = StratifiedKFold(n_splits=5,shuffle=True)
StratifiedKFoldResult =  list(skf.split(df1.iloc[:, :-1],df1.iloc[:, 21]))

In [55]:
list_k_means = [2,3,4,5,6,7,8,9,10,11,12,13,14,15]
acc_peso = {}

i = 0
sum = 0 
recall_true = 0
recall_false = 0
f1 = 0
precision_true = 0
precision_false = 0
f1_false = 0
tn_sum =0
fp_sum=0
fn_sum=0
tp_sum = 0
tempo_init = time.time()

for train_index, test_index in StratifiedKFoldResult :
  scaler = StandardScaler()
  scaler.fit(df1.iloc[:, :-1])

  train = df1.iloc[train_index]
  train.iloc[:, :-1] = scaler.transform(train.iloc[:, :-1])

  test = df1.iloc[test_index]
  test.iloc[:, :-1] = scaler.transform(test.iloc[:, :-1])

  x_df1,y_df1,x_df2,y_df2 = separar_dados(train,'true')

  sum_of_squared_distances_1 = []
  sum_of_squared_distances_2 = []

  for k in list_k_means:
    km1 = KMeans(n_clusters=k)
    km1 = km1.fit(x_df1)
    sum_of_squared_distances_1.append(km1.inertia_)

    km2 = KMeans(n_clusters=k)
    km2 = km2.fit(x_df2)
    sum_of_squared_distances_2.append(km2.inertia_)

  optimal_k1 = optimal_number_of_clusters(sum_of_squared_distances_1)
  optimal_k2 = optimal_number_of_clusters(sum_of_squared_distances_2)

  km1 = KMeans(n_clusters=optimal_k1)
  km1 = km1.fit(x_df1)

  new_y_df1 = km1.labels_

  km2 = KMeans(n_clusters=optimal_k2)
  km2 = km2.fit(x_df2)

  new_y_df2 = km2.labels_ + max(new_y_df1) + 1

  x_train = np.concatenate((x_df1,x_df2),axis=0)
  y_train = np.concatenate((new_y_df1,new_y_df2),axis=0)

  x_test = np.array(test.iloc[:, :-1].values)
  y_test = np.array(test.iloc[:, 21].values)

  gnb = GaussianNB()

  y_pred = gnb.fit(x_train,y_train).predict(x_test)

  new_pred = []
  for j in range(len(y_pred)):
    if(y_pred[j] in new_y_df1):
      new_pred.append(y_df1[0])
    else:
      new_pred.append(y_df2[0])

  print(' best k1:',optimal_k1,' best k2:',optimal_k2)
  print(classification_report(y_test, new_pred))

  tn, fp, fn, tp = confusion_matrix(y_test, new_pred).ravel() 

  tn_sum += tn
  fp_sum += fp
  fn_sum += fn
  tp_sum += tp

  sum += classification_report(y_test, new_pred,output_dict=True)['accuracy'] 
  recall_true += classification_report(y_test, new_pred,output_dict=True)['true']['recall']
  precision_true += classification_report(y_test, new_pred,output_dict=True)['true']['precision']
  f1 += f1_score(y_test, new_pred, average='binary',pos_label="true")

  recall_false += classification_report(y_test, new_pred,output_dict=True)['false']['recall']
  precision_false += classification_report(y_test, new_pred,output_dict=True)['false']['precision']

tempo = time.time() - tempo_init 
acc_peso[0] = [sum/5,0,"k-Means + Naive Bayes",recall_true/5,precision_true/5,recall_false/5,precision_false/5,tempo/5,tn_sum, fn_sum, tp_sum, fp_sum,f1/5]



 best k1: 4  best k2: 6
              precision    recall  f1-score   support

       false       0.82      0.95      0.88        83
        true       0.56      0.23      0.32        22

    accuracy                           0.80       105
   macro avg       0.69      0.59      0.60       105
weighted avg       0.77      0.80      0.77       105

 best k1: 5  best k2: 6
              precision    recall  f1-score   support

       false       0.85      0.96      0.90        83
        true       0.73      0.36      0.48        22

    accuracy                           0.84       105
   macro avg       0.79      0.66      0.69       105
weighted avg       0.83      0.84      0.82       105

 best k1: 6  best k2: 6
              precision    recall  f1-score   support

       false       0.86      0.86      0.86        83
        true       0.43      0.43      0.43        21

    accuracy                           0.77       104
   macro avg       0.64      0.64      0.64       104
we

In [56]:
i = 0
sum = 0 
recall_true = 0
recall_false = 0
f1= 0
precision_true = 0
precision_false = 0
f1_false = 0
tn_sum =0
fp_sum=0
fn_sum=0
tp_sum = 0
tempo_init = time.time()

for train_index, test_index in  StratifiedKFoldResult:
  scaler = StandardScaler()
  scaler.fit(df1.iloc[:, :-1])

  train = df1.iloc[train_index]
  train.iloc[:, :-1] = scaler.transform(train.iloc[:, :-1])

  test = df1.iloc[test_index]
  test.iloc[:, :-1] = scaler.transform(test.iloc[:, :-1])

  x_train = np.array(train.iloc[:, :-1].values)
  y_train = np.array(train.iloc[:, 21].values)

  x_test = np.array(test.iloc[:, :-1].values)
  y_test = np.array(test.iloc[:, 21].values)

  gnb = GaussianNB()

  new_pred = gnb.fit(x_train,y_train).predict(x_test)

  print(' best k1:',optimal_k1,' best k2:',optimal_k2)
  print(classification_report(y_test, new_pred))

  tn, fp, fn, tp = confusion_matrix(y_test, new_pred).ravel() 

  tn_sum += tn
  fp_sum += fp
  fn_sum += fn
  tp_sum += tp

  sum += classification_report(y_test, new_pred,output_dict=True)['accuracy'] 
  recall_true += classification_report(y_test, new_pred,output_dict=True)['true']['recall']
  precision_true += classification_report(y_test, new_pred,output_dict=True)['true']['precision']
  f1 += f1_score(y_test, new_pred, average='binary',pos_label="true")

  recall_false += classification_report(y_test, new_pred,output_dict=True)['false']['recall']
  precision_false += classification_report(y_test, new_pred,output_dict=True)['false']['precision']

tempo = time.time() - tempo_init 
acc_peso[1] = [sum/5,0,"Naive Bayes",recall_true/5,precision_true/5,recall_false/5,precision_false/5,tempo/5,tn_sum, fn_sum, tp_sum, fp_sum,f1/5]



 best k1: 5  best k2: 6
              precision    recall  f1-score   support

       false       0.88      0.90      0.89        83
        true       0.60      0.55      0.57        22

    accuracy                           0.83       105
   macro avg       0.74      0.72      0.73       105
weighted avg       0.82      0.83      0.83       105

 best k1: 5  best k2: 6
              precision    recall  f1-score   support

       false       0.88      0.95      0.91        83
        true       0.73      0.50      0.59        22

    accuracy                           0.86       105
   macro avg       0.81      0.73      0.75       105
weighted avg       0.85      0.86      0.85       105

 best k1: 5  best k2: 6
              precision    recall  f1-score   support

       false       0.85      0.94      0.89        83
        true       0.58      0.33      0.42        21

    accuracy                           0.82       104
   macro avg       0.72      0.64      0.66       104
we

In [57]:
i = 0
sum = 0 
recall_true = 0
recall_false = 0
f1 = 0
precision_true = 0
precision_false = 0
f1_false = 0
tn_sum =0
fp_sum=0
fn_sum=0
tp_sum = 0
tempo_init = time.time()

for train_index, test_index in  StratifiedKFoldResult:
  scaler = StandardScaler()
  scaler.fit(df1.iloc[:, :-1])

  train = df1.iloc[train_index]
  train.iloc[:, :-1] = scaler.transform(train.iloc[:, :-1])

  test = df1.iloc[test_index]
  test.iloc[:, :-1] = scaler.transform(test.iloc[:, :-1])

  x_train = np.array(train.iloc[:, :-1].values)
  y_train = np.array(train.iloc[:, 21].values)

  x_test = np.array(test.iloc[:, :-1].values)
  y_test = np.array(test.iloc[:, 21].values)

  knn = KNeighborsClassifier(n_neighbors=1)

  new_pred = knn.fit(x_train,y_train).predict(x_test)

  tn, fp, fn, tp = confusion_matrix(y_test, new_pred).ravel() 

  tn_sum += tn
  fp_sum += fp
  fn_sum += fn
  tp_sum += tp

  sum += classification_report(y_test, new_pred,output_dict=True)['accuracy'] 
  recall_true += classification_report(y_test, new_pred,output_dict=True)['true']['recall']
  precision_true += classification_report(y_test, new_pred,output_dict=True)['true']['precision']
  f1 += f1_score(y_test, new_pred, average='binary',pos_label="true")

  recall_false += classification_report(y_test, new_pred,output_dict=True)['false']['recall']
  precision_false += classification_report(y_test, new_pred,output_dict=True)['false']['precision']
tempo = time.time() - tempo_init 
acc_peso[2] = [sum/5,0,"K-NN",recall_true/5,precision_true/5,recall_false/5,precision_false/5,tempo/5,tn_sum, fn_sum, tp_sum, fp_sum,f1/5]

In [58]:
df = pd.DataFrame.from_dict(acc_peso,orient='index',columns=['taxa de acerto','valor de k','k-NN',"recall true","precision true","recall false","precision false","tempo teste","True Negative", "False Negative", "True Positive", "False Positive", "F1-Mesure"])

In [59]:
df

,taxa de acerto,valor de k,k-NN,recall true,precision true,recall false,precision false,tempo teste,True Negative,False Negative,True Positive,False Positive,F1-Mesure
0,0.806465,0,k-Means + Naive Bayes,0.375325,0.561829,0.918072,0.851109,1.540155,381,67,40,34,0.438629
1,0.833297,0,Naive Bayes,0.418615,0.645455,0.939759,0.863200,0.049819,390,62,45,25,0.502902
2,0.795000,0,K-NN,0.494372,0.500771,0.872289,0.870729,0.045561,362,54,53,53,0.494231
